### End-to-end pipeline to split a video

In [1]:
from src.bouldering.media.video.video import Video

In [2]:
# 1. Read the input video
VIDEO_PATH = "../data/video.mp4"

video = Video.read(VIDEO_PATH)
print(video.metadata)

{'video': {'fps': 25.0, 'n_frames': 10500, 'resolution': (1280, 720)}, 'audio': {'sample_rate': 44100, 'duration': 420.0025396825397}}


In [3]:
# 2. Create an OCR splitter

from src.bouldering.vision.ocr.splitter import SceneSplitterOCR
from src.bouldering.vision.scenes.ocr import OCRSceneSplitter

ocr_logic = SceneSplitterOCR(
    langs=["en"],
    crop_box=[0, 0.5, 0.5, 1],  # <-- relative coords
    fx=0.5,
    fy=0.5,
    stride=3,
    batch_size=16,
    smooth_window=5,
    majority_ratio=0.6,
    require_number=True,
)

ocr_splitter = OCRSceneSplitter(ocr_logic)

Using CPU. Note: This module is much faster with a GPU.


In [4]:
# 3. Create a scene splitter

from src.bouldering.vision.scenes.content import ContentSceneSplitter

content_splitter = ContentSceneSplitter(
    threshold=27.0,
    min_scene_len_sec=1.5,
    downscale=2,
)

/home/souheil_laptop/anaconda3/envs/altaroad310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# 4. Run the pipeline

from src.bouldering.vision.scenes.pipeline import ScenePipeline

pipeline = ScenePipeline(
    macro_splitter=ocr_splitter,
    micro_splitter=content_splitter,
)

scenes = pipeline.run(video.cut(0.0, 20.0))

len(scenes)

OCR scene splitting:   6%|███▎                                                     | 29/500 [00:00<00:03, 146.61frame/s]/home/souheil_laptop/anaconda3/envs/altaroad310/lib/python3.10/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
OCR scene splitting: 100%|█████████████████████████████████████████████████████████| 500/500 [02:43<00:00,  3.06frame/s]


0

In [6]:
scenes

[]